<a href="https://colab.research.google.com/github/fazeci/QA_with_rag/blob/main/QA_with_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import datetime
import numpy as np

# Normal dağılım kullanarak yanıt boyutunu üretme
def generate_normal_distribution_size(mean, std_dev):
    return max(0, int(np.random.normal(mean, std_dev)))

# Apache log formatına uygun sahte veriler oluşturma
def generate_fake_apache_log_entry():
    ip = ".".join(map(str, random.sample(range(1, 256), 4)))
    timestamp = datetime.datetime.now()
    method = random.choice(['GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'OPTIONS', 'PATCH', 'CONNECT', 'TRACE'])
    uri = random.choice(['/index.html', '/about.html', '/contact.html', '/login', '/products', '/api/data', '/admin'])
    protocol = "HTTP/1.1"
    status = random.choice([200, 201, 204, 301, 302, 304, 400, 401, 403, 404, 500, 502, 503, 504])
    size = generate_normal_distribution_size(mean=1500, std_dev=500)

    log_entry = f'{ip} - - [{timestamp.strftime("%d/%b/%Y:%H:%M:%S +0000")}] "{method} {uri} {protocol}" {status} {size}'
    return log_entry

# Belirli bir sayıda sahte log verisi oluşturma ve dosyaya yazma
def generate_and_save_fake_logs(num_entries, filename='fake_apache_logs.txt'):
    fake_logs = [generate_fake_apache_log_entry() for _ in range(num_entries)]

    with open(filename, 'w') as file:
        for entry in fake_logs:
            file.write(entry + '\n')

# Kullanım
generate_and_save_fake_logs(100000)

# İlk 10 log verisini görüntüleme
with open('fake_apache_logs.txt', 'r') as file:
    logs_to_display = [next(file) for _ in range(10)]
    print("".join(logs_to_display))


186.139.189.143 - - [14/Aug/2024:21:08:23 +0000] "TRACE /login HTTP/1.1" 304 1451
24.124.101.175 - - [14/Aug/2024:21:08:23 +0000] "OPTIONS /index.html HTTP/1.1" 302 801
235.246.208.77 - - [14/Aug/2024:21:08:23 +0000] "DELETE /login HTTP/1.1" 500 876
97.136.75.32 - - [14/Aug/2024:21:08:23 +0000] "TRACE /index.html HTTP/1.1" 302 1070
43.156.89.136 - - [14/Aug/2024:21:08:23 +0000] "DELETE /index.html HTTP/1.1" 302 1764
249.115.84.182 - - [14/Aug/2024:21:08:23 +0000] "CONNECT /contact.html HTTP/1.1" 401 1931
113.122.67.154 - - [14/Aug/2024:21:08:23 +0000] "CONNECT /login HTTP/1.1" 403 1766
224.137.73.28 - - [14/Aug/2024:21:08:23 +0000] "OPTIONS /index.html HTTP/1.1" 302 1570
31.134.59.129 - - [14/Aug/2024:21:08:23 +0000] "PATCH /index.html HTTP/1.1" 304 1632
13.87.200.14 - - [14/Aug/2024:21:08:23 +0000] "CONNECT /index.html HTTP/1.1" 400 1103



In [ ]:
import pandas as pd
import re

# Log dosyasını oku
log_file_path = '/content/fake_apache_logs.txt'

# Apache/Nginx log formatını regex ile ayıklama
log_pattern = r'(?P<ip>\d+\.\d+\.\d+\.\d+)\s+\S+\s+\S+\s+\[(?P<timestamp>.*?)\]\s+"(?P<method>\S+)\s+(?P<uri>\S+)\s+(?P<protocol>\S+)"\s+(?P<status>\d+)\s+(?P<size>\d+)'

# Logları DataFrame'e yükle
log_df = pd.read_csv(log_file_path, sep='|', names=['log_entry'])
log_df = log_df['log_entry'].str.extract(log_pattern)

# Temel analiz
print(log_df.head())
print(log_df.info())


                ip                   timestamp   method          uri  \
0  186.139.189.143  14/Aug/2024:21:08:23 +0000    TRACE       /login   
1   24.124.101.175  14/Aug/2024:21:08:23 +0000  OPTIONS  /index.html   
2   235.246.208.77  14/Aug/2024:21:08:23 +0000   DELETE       /login   
3     97.136.75.32  14/Aug/2024:21:08:23 +0000    TRACE  /index.html   
4    43.156.89.136  14/Aug/2024:21:08:23 +0000   DELETE  /index.html   

   protocol status  size  
0  HTTP/1.1    304  1451  
1  HTTP/1.1    302   801  
2  HTTP/1.1    500   876  
3  HTTP/1.1    302  1070  
4  HTTP/1.1    302  1764  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ip         100000 non-null  object
 1   timestamp  100000 non-null  object
 2   method     100000 non-null  object
 3   uri        100000 non-null  object
 4   protocol   100000 non-null  object
 5   status     

In [ ]:
# Veri türlerini dönüştürme
log_df['timestamp'] = pd.to_datetime(log_df['timestamp'], format='%d/%b/%Y:%H:%M:%S +0000')
log_df['status'] = log_df['status'].astype(int)
log_df['size'] = log_df['size'].astype(int)

print(log_df.dtypes)


ip                   object
timestamp    datetime64[ns]
method               object
uri                  object
protocol             object
status                int64
size                  int64
dtype: object


In [ ]:
# Geçersiz (olumsuz) status kodları veya size değerlerini kontrol etme
invalid_status = log_df[~log_df['status'].between(100, 599)]  # HTTP status kodları 100-599 aralığındadır
invalid_size = log_df[log_df['size'] < 0]  # Boyutun negatif olmaması gerekir

print(f"Geçersiz status kayıtları: {len(invalid_status)}")
print(f"Geçersiz size kayıtları: {len(invalid_size)}")


Geçersiz status kayıtları: 0
Geçersiz size kayıtları: 0


In [ ]:
# En sık kullanılan HTTP metodlarını analiz etme
method_counts = log_df['method'].value_counts()
print(method_counts)


method
GET        11257
TRACE      11245
PUT        11202
HEAD       11146
DELETE     11104
POST       11081
CONNECT    11042
PATCH      10976
OPTIONS    10947
Name: count, dtype: int64


In [ ]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from sentence_transformers import SentenceTransformer

# Modeli yükle
model = SentenceTransformer('all-MiniLM-L6-v2')

# Log verisini birleştirerek metin formatında kullanıma hazırlama
log_df['log_text'] = log_df.apply(lambda row: f"{row['method']} {row['uri']} {row['status']}", axis=1)

# Vektörlere dönüştürme
log_vectors = model.encode(log_df['log_text'].tolist(), show_progress_bar=True)

print(log_vectors.shape)  # Vektörlerin boyutunu kontrol et


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

(100000, 384)


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.0 MB/s eta 0:00:00


In [ ]:
import faiss

# Vektörlerin boyutunu al
vector_dim = log_vectors.shape[1]

# FAISS index'i oluştur (L2 norm kullanarak)
index = faiss.IndexFlatL2(vector_dim)

# Vektörleri index'e ekle
index.add(np.array(log_vectors))

# Örnek bir sorgu yapalım: ilk logun vektörü ile en benzer 4 logu bulalım
D, I = index.search(log_vectors[:1], k=4)  # D: mesafeler, I: indexler

# En benzer log kayıtlarını göster
print("En benzer log kayıtları:")
print(log_df.iloc[I[0]])


En benzer log kayıtları:
                   ip           timestamp method     uri  protocol  status  \
0     186.139.189.143 2024-08-14 21:08:23  TRACE  /login  HTTP/1.1     304   
2139    181.28.78.243 2024-08-14 21:08:23  TRACE  /login  HTTP/1.1     304   
3565      25.4.86.102 2024-08-14 21:08:23  TRACE  /login  HTTP/1.1     304   
5874  237.208.226.158 2024-08-14 21:08:23  TRACE  /login  HTTP/1.1     304   

      size          log_text  
0     1451  TRACE /login 304  
2139  1694  TRACE /login 304  
3565  1721  TRACE /login 304  
5874  1273  TRACE /login 304  


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# T5 modelini yükle
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def retrieve_similar_logs(query_vector, index, k=6):
    """
    Kullanıcıdan gelen sorgu vektörü ile benzer log kayıtlarını bulur.

    Args:
    query_vector: Sorgu metninin vektör formatı
    index: FAISS vektör veri tabanı indeksi
    k: En benzer kaç kayıt bulunacağı

    Returns:
    En benzer log kayıtlarının DataFrame'i
    """
    D, I = index.search(query_vector, k=k)

    # En benzer log kayıtlarını DataFrame'e dönüştürme
    response = log_df.iloc[I[0]]

    return response

query_text = input("Sorgunuzu girin: ")

# Sorgu metnini vektör haline getirme
query_vector = model.encode([query_text])

# Benzer logları bulma
response1 = retrieve_similar_logs(query_vector, index)

print(response1)


def generate_response(query_text, response1):
    """

    Args:
    query_text: Kullanıcıdan gelen sorgu metni
    response1: retrievedan gelen log kayıtları

    Returns:
    Jeneratif model tarafından oluşturulan yanıt
    """

    # Log kayıtlarını metin formatında birleştirme
    context = " ".join(response1['log_text'].tolist())

    # T5 modeli için giriş hazırlama
    input_text = f"Based on the following logs: {context}, please provide a detailed response to the user query: {query_text}. The response should reflect the most common pattern and status codes seen in the logs."
    input_ids = t5_tokenizer.encode(input_text, return_tensors='pt')
    # Yanıt oluşturma (tekrarı önlemek için no_repeat_ngram_size kullanımı)
    outputs = t5_model.generate(
        input_ids,
        max_new_tokens=150,
        no_repeat_ngram_size=2,  # N-gram tekrarlarını önlemek için
        num_beams=5,             # Daha iyi yanıt için beam search kullanımı
    )
    response = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

response2 = generate_response(query_text, response1)

print(f"Sorgu: {query_text}\nYanıt: {response2}")


def rag_model(query_text):
    """
    Args:
    query_text: Kullanıcı sorgusu (metin)

    Returns:
    Model tarafından üretilen yanıt (metin)
    """
    # Sorgu metnini vektör haline getirme
    query_vector = model.encode([query_text])

    # Yanıt oluşturma
    response = generate_response(query_text, response1)

    return response

response3 = rag_model(query_text)

print(f"Sorgu: {query_text}\nYanıt: {response3}")


Sorgunuzu girin: POST /login 403
                   ip           timestamp method     uri  protocol  status  \
145   198.205.123.109 2024-08-14 21:08:23   POST  /login  HTTP/1.1     403   
1788  106.104.129.247 2024-08-14 21:08:23   POST  /login  HTTP/1.1     403   
3516  144.199.216.152 2024-08-14 21:08:23   POST  /login  HTTP/1.1     403   
4588  249.162.111.213 2024-08-14 21:08:23   POST  /login  HTTP/1.1     403   
4979   216.59.128.196 2024-08-14 21:08:23   POST  /login  HTTP/1.1     403   
6383    216.121.25.34 2024-08-14 21:08:23   POST  /login  HTTP/1.1     403   

      size         log_text  
145   1708  POST /login 403  
1788  1841  POST /login 403  
3516   964  POST /login 403  
4588   907  POST /login 403  
4979  1586  POST /login 403  
6383  2542  POST /login 403  
Sorgu: POST /login 403
Yanıt: POST /login 403.
Sorgu: POST /login 403
Yanıt: POST /login 403.


In [ ]:
query1 = "DELETE /contact.html 401"
answer1 = rag_model(query1)
print(similar_logs)
print(response)
print(f"Sorgu: {query1}\nYanıt: {answer1}")

query2 = "GET /about.html 200"
answer2 = rag_model(query2)
print(similar_logs)
print(response)
print(f"Sorgu: {query2}\nYanıt: {answer2}")

query3 = "POST /login 403"
answer3 = rag_model(query3)
print(similar_logs)
print(response)
print(f"Sorgu: {query3}\nYanıt: {answer3}")

query4 = "GET /api/data 200"
answer4 = rag_model(query4)
print(similar_logs)
print(response)
print(f"Sorgu: {query4}\nYanıt: {answer4}")

query5 = "GET /index.html 200"
answer5 = rag_model(query5)
print(similar_logs)
print(response)
print(f"Sorgu: {query5}\nYanıt: {answer5}")



**Proje Raporu: Log Verileri Üzerinde RAG Modeli ile Sorgu Yanıtlama Sistemi**

***1. Giriş***

*Projenin Amacı:*
Bu proje, web trafik log verilerini kullanarak kullanıcı sorgularına uygun yanıtlar veren bir Retrieval-Augmented Generation (RAG) sistemi geliştirmeyi amaçlamaktadır. Sistem, log verileri üzerinde arama yaparak en uygun kayıtları bulmakta ve bu kayıtları kullanarak jeneratif bir dil modeli ile kullanıcıya yanıt oluşturmaktadır.

*Kullanılan Veri Seti:*
- Apache log verileri
- 100,000 satırlık örnek log kayıtları (IP adresleri, erişilen sayfalar, zaman damgaları vb.)

***2. Metodoloji***

*Aşama 1: Veri Hazırlığı ve Ön İşleme*
- Veri Analizi: Log verileri incelendi ve gerekli sütunlar (IP adresi, URI, HTTP metodu, durum kodu vb.) ayıklandı.
- Veri Temizleme: Eksik veya hatalı veriler temizlendi ve sütunlar uygun veri türlerine dönüştürüldü.
- Vektörleştirme: Log kayıtları, dil modeli kullanılarak vektörlere dönüştürüldü. FAISS kütüphanesi ile bu vektörler sorgulanabilir bir veri tabanına yüklendi.

*Aşama 2: RAG Modelinin Kurulumu*
- Bilgi Alma Modülü: FAISS kütüphanesi kullanılarak, kullanıcıdan gelen sorguya en benzer log kayıtlarını bulmak üzere bir bilgi alma modülü oluşturuldu.
- Jeneratif Modül:`T5` modelini kullanarak, bulunan log kayıtları üzerinde anlamlı yanıtlar oluşturmak için bir jeneratif modül geliştirildi.
- Sistem Entegrasyonu: Bilgi alma ve jeneratif modülleri entegre edilerek, tam işlevsel bir RAG modeli oluşturuldu.

*Aşama 3: Sistem Entegrasyonu ve Test*
- Sistemin Test Edilmesi: Çeşitli kullanıcı sorguları ile sistem test edildi. Yanıtların doğruluğu ve kalitesi değerlendirildi.
- Sistem Performansı: Modelin performansı, sorgu süresi ve yanıt kalitesi açısından ölçüldü.

***3. Karşılaşılan Zorluklar ve Çözümleri***

- Zorluk: Log verilerinin karmaşıklığı ve tekrarlayan bilgiler.
  
  Çözüm:Veriyi vektörleştirme sürecinde, benzer log kayıtlarını daha iyi işleyebilmek için veri temizliği ve özetleme teknikleri uygulandı.
  
- Zorluk: Modelin tekrarlayan veya anlamsız yanıtlar üretmesi.
  
  Çözüm: Modelde `no_repeat_ngram_size` ve `max_new_tokens` parametreleri ayarlandı. Ayrıca, giriş kontekstinin daha anlamlı hale getirilmesi sağlandı.

- Zorluk: Veri Türleri ile İlgili Sorunlar
  
  Çözüm: Veri türlerini uygun hale getirmek için öncelikle her bir sütunun içeriği analiz edildi ve uygun veri türlerine dönüştürüldü.
  
Zaman Damgaları: timestamp sütunu, datetime formatına dönüştürüldü, böylece zaman bazlı gruplama ve analizler yapılabildi.

Durum Kodları ve Yanıt Boyutları: status ve size sütunları, int64 veri türüne dönüştürüldü, böylece sayısal işlemler ve karşılaştırmalar yapılabildi.

IP Adresleri: IP adresleri, string formatında bırakıldı, ancak gerektiğinde IP’lerin sınıflandırılması ve gruplandırılması için düzenli ifadeler (regex) kullanıldı.

***4. Performans Değerlendirmesi***

-Doğruluk: Sistem, kullanıcı sorgularına yanıt verirken doğru ve alakalı log kayıtlarını kullandı. Yanıtlar, log kayıtlarının genel paternlerine uygun şekilde oluşturuldu.
-Yanıt Süresi: Sistem, büyük veri setiyle çalışmasına rağmen makul sürelerde yanıt üretebildi. FAISS vektör veri tabanı, yüksek verimli arama sağladı.
- Yanıt Kalitesi: Model, bazı durumlarda tekrarlayan veya eksik yanıtlar üretse de, yapılan optimizasyonlarla daha tutarlı yanıtlar alınmaya başlandı.

***5. Sonuç ve Öneriler***

* Genel Değerlendirme: Geliştirilen sistem, log verileri üzerinde başarılı bir
şekilde sorgu yapabilen ve anlamlı yanıtlar üretebilen bir RAG modeli olarak çalıştı. Performans açısından yüksek verimlilik sağlandı.
* Öneriler: Daha farklı veri türleriyle sistemin test edilmesi, sistemin genelleme yeteneğini artırabilir.

***6. Kaynaklar***

- Kullanılan kütüphaneler: `transformers`, `sentence-transformers`, `faiss`, `pandas`, `numpy`



FATMA ZEHRA ÇINAR
